In [12]:
import pandas as pd
import os
import mlflow
import sys
sys.path.append('src')
from src.models.train_model_svd import *
from src.models.load_svd_data import *


In [13]:
svd_model = load_svd_model()

In [35]:
def collab_reco(user_id, svd_model, df_ratings, num_recommendations=10):
 
    # Calculer la moyenne globale des notes
    df_ratings= df_ratings[["userId","movieId","rating"]]
    moyenne = df_ratings['rating'].mean()
    df_movies = pd.read_csv("../src/data/raw/movies.csv")
    # Obtenir les films notés par l'utilisateur
    user_rated_movies = set(df_ratings[df_ratings['userId'] == user_id]['movieId'])
    
    # Créer la liste anti_testset pour les films que l'utilisateur n'a pas notés
    all_movies = set(df_movies['movieId'])
    anti_testset = [(user_id, movie_id, moyenne) for movie_id in all_movies if movie_id not in user_rated_movies]
    
    # Generate predictions using the SVD model
    predictions_svd = svd_model.test(anti_testset)
    predictions_svd = pd.DataFrame(predictions_svd)

    # Load movie data to map movie IDs to titles
    movieId_title_map = df_movies.set_index('movieId')['title'].to_dict()
    predictions_svd['title'] = predictions_svd['iid'].map(movieId_title_map)

    # Rename and reorder the columns for clarity
    predictions_svd = predictions_svd.rename(columns={'uid': 'userId', 'est': 'note'})
    predictions_svd = predictions_svd[['userId', 'title', 'note']]
    # Sort the predictions by rating in descending order
    predictions_svd.sort_values('note', ascending=False, inplace=True)

    return predictions_svd.head(num_recommendations)

In [33]:
df_ratings = pd.read_csv("../src/data/raw/ratings.csv")

In [37]:
recommendations = collab_reco(user_id=123, svd_model=svd_model, df_ratings=df_ratings, num_recommendations=10)
print(recommendations)

       userId                                   title      note
15348     123             Ween Live in Chicago (2004)  4.544851
284       123                     Pulp Fiction (1994)  4.452744
17650     123                          Welfare (1975)  4.452332
19278     123                          The War (2007)  4.446491
17770     123         Great White Silence, The (1924)  4.397552
11097     123                     Edvard Munch (1974)  4.383126
824       123                   Godfather, The (1972)  4.346369
21811     123         Century of the Self, The (2002)  4.315091
15253     123                           Cosmos (1980)  4.298514
15742     123  Captain Conan (Capitaine Conan) (1996)  4.280695


In [30]:
df_ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [31]:
df_ratings["rating"].mean()

3.5255285642993797